In [ ]:
!pip install tvm_valuetypes

In [1]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM, method_name_to_id, code_disasseble
import codecs
from tvm_valuetypes import Cell, deserialize_boc

# Just code

In [2]:
PyTVM(code="te6ccgEBAQEAAwAAAnE=").run_vm()

[1]

In [3]:
SUB = "te6ccgEBAQEAAwAAAqE="
a = 3
b = 1

tvm = PyTVM(code=SUB)
tvm.set_stack([a, b])
tvm.run_vm()

[2]

# Disassembler

In [4]:
code_disasseble("te6ccgEBAQEAAwAAAnE=")

'PUSHINT 1\n'

# Easy test examples

In [5]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Use code: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:10.647932][arithops.cpp:34]	execute PUSHINT 1

[ 4][t 0][2022-10-21 16:23:10.648107][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0
Got 1 from TVM: 1


 [ ] 
 [ 1 ] 


In [6]:
tvm_result

[1]

In [7]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()
tvm_result

[1,
 {'type': 'continuation',
  'value': 'te6ccuEBAgEADgASHAEMBAAAAAYAAQAGce1DB7kzoA=='}]

In [8]:
cell = Cell()
b64 = tvm_result[1]['value']
boc = codecs.decode(codecs.encode(b64, 'utf8'), 'base64')
deserialize_boc(boc)

<Cell refs_num: 1, data: b'\x04\x00\x00\x00\x06\x00'>

In [9]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

[124711402,
 0,
 0,
 0,
 0,
 0,
 0,
 [100000, None],
 {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
 None]

## Change c7

In [10]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [11]:
tvm.set_c7(translt=100,
           unixtime=179, 
           globalConfig="te6ccgEBAQEABAAABO1H",
           randseed=100, 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=100000000)

In [12]:
tvm.run_vm()

[[124711402,
  0,
  0,
  179,
  0,
  100,
  100,
  [100000000, None],
  {'type': 'cellSlice',
   'value': 'te6ccuEBAQEAJABIAEOADVySWN59rHc58b7yQKDOcZsaNW4TYZQKwLQsP8G0qTtQ5qaveg=='},
  {'type': 'cell', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='}]]

## Get detailed result

In [13]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,
"\nVM success: ",tvm.success)

Exit code:  -1 
C5:  te6ccuEBAQEAAgAEAABmLc6k 
Gas credit:  0 
Gas used:  31 
C4:  te6ccuEBAQEAAgAEAABmLc6k 
VM Steps:  2 
VM final hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM init hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM success:  True


## Set stack

In [14]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H", log_level=6)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C


In [15]:
# actually python3 & TON have really big integers :))
tvm.set_stack([
    190009329293929392939239293293929329392949949949299008888888880999999888, 
    {'type': "cell",
     'value': 'te6ccgEBAQEABAAABO1H'}, 
    [{'type': "cellSlice",
     'value': 'te6ccgEBAQEABAAABO1H'},
     12312312313213132,
     [1231232132, 1321231231, 57, 179]
    ]])

In [16]:
tvm.run_vm()

DEBUG: Use code: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:20.758002][contops.cpp:694]	execute PUSH c7

[ 4][t 0][2022-10-21 16:23:20.758371][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #3
DEBUG: Parse stack item #2
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ 190009329293929392939239293293929329392949949949299008888888880999999888 C{2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C} [CS{Cell{0004ed47} bits: 0..16; refs: 0..0} 12312312313213132 [1231232132 1321231231 57 179]] ] 
 [ 190009329293929392939239293293929329392949949949299008888888880999999888 C{2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C} [CS{Cell{0004ed47} bits: 0..16; refs: 0..0} 12312312313213132 [1231232132 1321231231 57 179]] [124711402 0 0 0 0 0 0 (100000) CS{Cell{00100000000000000000} bits: 0..64; refs: 0..0} ()] ] 


[190009329293929392939239293293929329392949949949299008888888880999999888,
 {'type': 'cell', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
 [{'type': 'cellSlice', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
  12312312313213132,
  [1231232132, 1321231231, 57, 179]],
 [124711402,
  0,
  0,
  0,
  0,
  0,
  0,
  [100000, None],
  {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
  None]]

## C4 (Data) usage example

In [17]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [18]:
# You can see, that 179 was loaded from c4, and cellSlice on stack
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:21.387238][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-21 16:23:21.387441][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-21 16:23:21.387590][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-21 16:23:21.387702][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Gas Limit 

In [19]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [20]:
tvm.set_gasLimit(26) # You can also pass max gas set_gasLimit(gas_limit=26, gas_max=26)
tvm.run_vm() # out of gas

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:21.891406][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-21 16:23:21.893487][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-21 16:23:21.897457][vm.cpp:488]	unhandled out-of-gas exception: gas consumed=144, limit=26

DEBUG: VM terminated with exit code 13
DEBUG: Parse stack item #0


[144]

## Libs 

This is just load example, they will pass to VM, but I'm little bussy to get code of loading libs, check out [akifoq](https://github.com/akifoq) github

In [21]:
tvm.set_libs([{'type': 'cell', 'value': 'te6ccgEBAQEABwAACu1E0NM/'}])

## Stack dump log Level

In [22]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [23]:
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:22.907896][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-21 16:23:22.907979][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-21 16:23:22.908087][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-21 16:23:22.908188][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ ] 
 [ C{E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449} ] 
 [ CS{Cell{001000000000000000b3} bits: 0..64; refs: 0..0} ] 
 [ 179 CS{Cell{001000000000000000b3} bits: 64..64; refs: 0..0} ] 


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Allow debug & same c3

In [24]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4,
            allow_debug=True,
            same_c3=False)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [25]:
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:23.757686][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-21 16:23:23.757900][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-21 16:23:23.758114][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-21 16:23:23.758275][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ ] 
 [ C{E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449} ] 
 [ CS{Cell{001000000000000000b3} bits: 0..64; refs: 0..0} ] 
 [ 179 CS{Cell{001000000000000000b3} bits: 64..64; refs: 0..0} ] 


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Accout get methods

In [26]:
# get account code & data from dton.io
tvm = PyTVM(code="te6ccuECFAEAAtQAABoAJAAuATICIAIqAqYDNAM+A5wDpgO8A/4ECAQmBEQEtgUqBZoFqAEU/wD0pBP0vPLICwECASACAwIBSAQFBPjygwjXGCDTH9Mf0x8C+CO78mTtRNDTH9Mf0//0BNFRQ7ryoVFRuvKiBfkBVBBk+RDyo/gAJKTIyx9SQMsfUjDL/1IQ9ADJ7VT4DwHTByHAAJ9sUZMg10qW0wfUAvsA6DDgIcAB4wAhwALjAAHAA5Ew4w0DpMjLHxLLH8v/EBESEwLm0AHQ0wMhcbCSXwTgItdJwSCSXwTgAtMfIYIQcGx1Z70ighBkc3RyvbCSXwXgA/pAMCD6RAHIygfL/8nQ7UTQgQFA1yH0BDBcgQEI9ApvoTGzkl8H4AXTP8glghBwbHVnupI4MOMNA4IQZHN0crqSXwbjDQYHAgEgCAkAeAH6APQEMPgnbyIwUAqhIb7y4FCCEHBsdWeDHrFwgBhQBMsFJs8WWPoCGfQAy2kXyx9SYMs/IMmAQPsABgCKUASBAQj0WTDtRNCBAUDXIMgBzxb0AMntVAFysI4jghBkc3Rygx6xcIAYUAXLBVADzxYj+gITy2rLH8s/yYBA+wCSXwPiAgEgCgsAWb0kK29qJoQICga5D6AhhHDUCAhHpJN9KZEM5pA+n/mDeBKAG3gQFImHFZ8xhAIBWAwNABG4yX7UTQ1wsfgAPbKd+1E0IEBQNch9AQwAsjKB8v/ydABgQEI9ApvoTGACASAODwAZrc52omhAIGuQ64X/wAAZrx32omhAEGuQ64WPwABu0gf6ANTUIvkABcjKBxXL/8nQd3SAGMjLBcsCIs8WUAX6AhTLaxLMzMlz+wDIQBSBAQj0UfKnAgBwgQEI1xj6ANM/yFQgR4EBCPRR8qeCEG5vdGVwdIAYyMsFywJQBs8WUAT6AhTLahLLH8s/yXP7AAIAbIEBCNcY+gDTPzBSJIEBCPRZ8qeCEGRzdHJwdIAYyMsFywJQBc8WUAP6AhPLassfEss/yXP7AAAK9ADJ7VQ32i8B", 
            data="te6ccuEBBAEAegBYZKz0AVEAAABfKamjFwar2UnSv7yFJMcBHp8S/8es5VZ1DSsQduQyjHhdbU9awAECA8CIAgMAQ7/9RH6C0XtQV/uceTSwYGqZEYjcVUKe3D2prT8KNc4u5EAAQ7/Nan0TcqBqB20NuDhtNOk60TWn6tZwV5g88o9fm8cwK8CwqgZg", 
            log_level=4)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: FEB5FF6820E2FF0D9483E7E0D62C817D846789FB4AE580C878866D959DABD5C0
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E59BCE6701021EA2EA693DED7D9BB6D0193BE042C424DBD91DF5F7D5F752EFFD


In [27]:
tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams=100000000, # or trying get own balance from c7
           randseed=1) # or maybe you use random in your contract :)

In [28]:
method_name_to_id("get_seqno") # get methods have own numbers :)

77871

In [29]:
tvm.set_stack([method_name_to_id("seqno")]) # long story short, you need to pass method_id to stack

In [30]:
tvm.run_vm() # this is seqno of this wallet :)

DEBUG: Use code: FEB5FF6820E2FF0D9483E7E0D62C817D846789FB4AE580C878866D959DABD5C0
DEBUG: Load cp0
[ 4][t 0][2022-10-21 16:23:25.273947][contops.cpp:1129]	execute SETCP 0

[ 4][t 0][2022-10-21 16:23:25.274064][dictops.cpp:631]	execute DICTPUSHCONST 19 (xC_,1)

[ 4][t 0][2022-10-21 16:23:25.274225][dictops.cpp:595]	execute DICTIGETJMPZ

[ 4][t 0][2022-10-21 16:23:25.274466][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-21 16:23:25.274529][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-21 16:23:25.274615][cellops.cpp:1061]	execute PLDU 32

[ 4][t 0][2022-10-21 16:23:25.274672][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0


 [ 85143 ] 
 [ 85143 ] 
 [ 85143 C{D814641364BD13E0A8A646226D7EB6E174CFD4F47235F8F8B29D1FD68C05A399} 19 ] 
 [ ] 
 [ C{E59BCE6701021EA2EA693DED7D9BB6D0193BE042C424DBD91DF5F7D5F752EFFD} ] 
 [ CS{Cell{01510000005f29a9a31706abd949d2bfbc8524c7011e9f12ffc7ace556750d2b1076e4328c785d6d4f5ac0} bits: 0..321; refs: 0..1} ] 
 [ 95 ] 


[95]